In [112]:
import pandas as pd
import numpy as np

In [113]:
df = pd.read_csv('../data/process_v1/process_data_4.csv')

In [114]:
del df['unitPrice']

In [115]:
del df['landType']

In [116]:
df.head(3)

,numberOfFloors,numberOfBathRooms,numberOfBedRooms,numberOfKitchens,numberOfGarages,certificateOfLandUseRight,ward,street,district,city,...,lon,typeOfRealEstate,frontWidth,endWidth,facade,houseDirection,landSize,price,accessibility,description
0,1,NaN,NaN,NaN,NaN,no,hoài đức - vân canh,hoài đức - vân canh - hậu ái,hoài đức,hà nội,...,105.727235,privateLand,4.2,NaN,oneSideOpen,southeast,50.0,4.6,fitThreeCars,"bán đất dịch vụ 6,9ha vân canh, hoài đức, hà n..."
1,5,5.0,4.0,NaN,NaN,yes,hoàng mai - giáp bát,hoàng mai - giáp bát - giáp bát,hoàng mai,hà nội,...,105.843605,privateProperty,0.0,NaN,oneSideOpen,NaN,35.0,3.6,fitOneCarAndOneMotorbike,"+ cần bán nhà phố giáp bát, giáp tuyến phố kim..."
2,4,NaN,NaN,NaN,NaN,yes,hà đông - văn quán,hà đông - văn quán - chiến thắng,hà đông,hà nội,...,105.796775,privateProperty,4.5,4.5,oneSideOpen,NaN,40.0,3.9,notInTheAlley,"bán nhà giá bất ngờ 3.9 tỷ,có dt chính 40 m2 ,..."


In [117]:
df['w'] = df['frontWidth']
df['h'] = df['landSize'] / df['w']

In [118]:
df = df.replace(np.inf, np.nan)

In [119]:
df.columns.tolist()

['numberOfFloors',
 'numberOfBathRooms',
 'numberOfBedRooms',
 'numberOfKitchens',
 'numberOfGarages',
 'certificateOfLandUseRight',
 'ward',
 'street',
 'district',
 'city',
 'lat',
 'lon',
 'typeOfRealEstate',
 'frontWidth',
 'endWidth',
 'facade',
 'houseDirection',
 'landSize',
 'price',
 'accessibility',
 'description',
 'w',
 'h']

In [120]:
location_df = df[['lat', 'lon']].drop_duplicates()

In [121]:
facility_500_df = pd.read_csv('../feature_repo/src/feature_engineering/facility/data/table/hcm_hn_distance_500_facility_count.csv')
facility_1000_df = pd.read_csv('../feature_repo/src/feature_engineering/facility/data/table/hcm_hn_distance_1000_facility_count.csv')
facility_2000_df = pd.read_csv('../feature_repo/src/feature_engineering/facility/data/table/hcm_hn_distance_2000_facility_count.csv')

In [122]:
def rename_col(dataframe, distance):
    cols = [c for c in dataframe.columns if c  not in ['lat', 'lon']]
    rename_cols = [f'num_of_{col}_in_{distance}m_radius' for col in cols]

    rename_col_dict = dict(zip(cols, rename_cols))
    return dataframe.rename(columns = rename_col_dict)

facility_500_df = rename_col(facility_500_df, 500)
facility_1000_df = rename_col(facility_1000_df, 1000)
facility_2000_df = rename_col(facility_2000_df, 2000)


In [123]:
for facility_df in [facility_500_df, facility_1000_df, facility_2000_df]:
    df = df.merge(facility_df, how='left', on = ['lat', 'lon'])

In [124]:
df

,numberOfFloors,numberOfBathRooms,numberOfBedRooms,numberOfKitchens,numberOfGarages,certificateOfLandUseRight,ward,street,district,city,...,num_of_marketplace_in_2000m_radius,num_of_restaurant_in_2000m_radius,num_of_hospital_in_2000m_radius,num_of_school_in_2000m_radius,num_of_kindergarten_in_2000m_radius,num_of_townhall - community_centre_in_2000m_radius,num_of_police_in_2000m_radius,num_of_place_of_worship_in_2000m_radius,num_of_bank_in_2000m_radius,num_of_atm_in_2000m_radius
0,1,NaN,NaN,NaN,NaN,no,hoài đức - vân canh,hoài đức - vân canh - hậu ái,hoài đức,hà nội,...,0,3,1,11,3,0,2,18,0,0
1,5,5.0,4.0,NaN,NaN,yes,hoàng mai - giáp bát,hoàng mai - giáp bát - giáp bát,hoàng mai,hà nội,...,4,16,5,38,20,0,13,3,34,51
2,4,NaN,NaN,NaN,NaN,yes,hà đông - văn quán,hà đông - văn quán - chiến thắng,hà đông,hà nội,...,5,20,7,39,13,0,7,13,10,46
3,7,7.0,7.0,NaN,NaN,yes,hoàn kiếm - lý thái tổ,hoàn kiếm - lý thái tổ - lý thái tổ,hoàn kiếm,hà nội,...,15,694,25,52,25,0,38,80,128,466
4,4,NaN,NaN,NaN,NaN,yes,cầu giấy - nghĩa đô,cầu giấy - nghĩa đô - hoàng quốc việt,cầu giấy,hà nội,...,11,107,8,50,23,0,14,40,35,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170959,1,2.0,2.0,NaN,NaN,no,hai bà trưng - vĩnh tuy,hai bà trưng - vĩnh tuy - mạc thị bưởi,hai bà trưng,hà nội,...,4,11,6,41,24,0,14,5,13,58
170960,6,10.0,10.0,NaN,NaN,no,thanh xuân - thượng đình,thanh xuân - thượng đình - thượng đình,thanh xuân,hà nội,...,13,86,11,66,25,0,20,15,64,144
170961,8,22.0,22.0,NaN,NaN,yes,long biên - long biên,long biên - long biên - cổ linh,long biên,hà nội,...,2,39,1,9,4,0,4,13,1,7
170962,5,NaN,NaN,NaN,NaN,yes,nam từ liêm - mỹ đình 1,nam từ liêm - mỹ đình 1 - đình thôn,nam từ liêm,hà nội,...,5,24,6,31,10,0,2,12,10,21


In [125]:
num_cols = []
cat_cols = []
for c in df.columns.tolist():
    if df[c].dtype != 'object':
        num_cols.append(c)
    else:
        cat_cols.append(c)


In [126]:
cat_cols

['certificateOfLandUseRight',
 'ward',
 'street',
 'district',
 'city',
 'typeOfRealEstate',
 'facade',
 'houseDirection',
 'accessibility',
 'description']

In [127]:
num_cols

['numberOfFloors',
 'numberOfBathRooms',
 'numberOfBedRooms',
 'numberOfKitchens',
 'numberOfGarages',
 'lat',
 'lon',
 'frontWidth',
 'endWidth',
 'landSize',
 'price',
 'w',
 'h',
 'num_of_university_in_500m_radius',
 'num_of_fuel_in_500m_radius',
 'num_of_cafe_in_500m_radius',
 'num_of_parking_in_500m_radius',
 'num_of_parking_entrance_in_500m_radius',
 'num_of_fast_food_in_500m_radius',
 'num_of_marketplace_in_500m_radius',
 'num_of_restaurant_in_500m_radius',
 'num_of_hospital_in_500m_radius',
 'num_of_school_in_500m_radius',
 'num_of_kindergarten_in_500m_radius',
 'num_of_townhall - community_centre_in_500m_radius',
 'num_of_police_in_500m_radius',
 'num_of_place_of_worship_in_500m_radius',
 'num_of_bank_in_500m_radius',
 'num_of_atm_in_500m_radius',
 'num_of_university_in_1000m_radius',
 'num_of_fuel_in_1000m_radius',
 'num_of_cafe_in_1000m_radius',
 'num_of_parking_in_1000m_radius',
 'num_of_parking_entrance_in_1000m_radius',
 'num_of_fast_food_in_1000m_radius',
 'num_of_market

In [128]:
df[
'typeOfRealEstate'
].value_counts()

typeOfRealEstate
privateProperty         77386
townhouse               30638
condominium             29533
privateLand             19695
semiDetachedVilla       11563
shophouse                1500
otherTypesOfProperty      561
resort                     88
Name: count, dtype: int64

In [129]:
certificateOfLandUseRight_list = list(df['certificateOfLandUseRight'].unique())
certificateOfLandUseRight_dict = dict(zip(certificateOfLandUseRight_list, range(len(certificateOfLandUseRight_list))))
df['certificateOfLandUseRight'] = df['certificateOfLandUseRight'].map(certificateOfLandUseRight_dict)

ward_list = list(df['ward'].unique())
ward_dict = dict(zip(ward_list, range(len(ward_list))))
df['ward'] = df['ward'].map(ward_dict)
street_list = list(df['street'].unique())
street_dict = dict(zip(street_list, range(len(street_list))))
df['street'] = df['street'].map(street_dict)
district_list = list(df['district'].unique())
district_dict = dict(zip(district_list, range(len(district_list))))
df['district'] = df['district'].map(district_dict)
city_list = list(df['city'].unique())
city_dict = dict(zip(city_list, range(len(city_list))))
df['city'] = df['city'].map(city_dict)
typeOfRealEstate_list = list(df['typeOfRealEstate'].unique())
typeOfRealEstate_dict = dict(zip(typeOfRealEstate_list, range(len(typeOfRealEstate_list))))
df['typeOfRealEstate'] = df['typeOfRealEstate'].map(typeOfRealEstate_dict)
facade_list = list(df['facade'].unique())
facade_dict = dict(zip(facade_list, range(len(facade_list))))
df['facade'] = df['facade'].map(facade_dict)
houseDirection_list = list(df['houseDirection'].unique())
houseDirection_dict = dict(zip(houseDirection_list, range(len(houseDirection_list))))
df['houseDirection'] = df['houseDirection'].map(houseDirection_dict)
accessibility_list = list(df['accessibility'].unique())
accessibility_dict = dict(zip(accessibility_list, range(len(accessibility_list))))
df['accessibility'] = df['accessibility'].map(accessibility_dict)

In [130]:
df

,numberOfFloors,numberOfBathRooms,numberOfBedRooms,numberOfKitchens,numberOfGarages,certificateOfLandUseRight,ward,street,district,city,...,num_of_marketplace_in_2000m_radius,num_of_restaurant_in_2000m_radius,num_of_hospital_in_2000m_radius,num_of_school_in_2000m_radius,num_of_kindergarten_in_2000m_radius,num_of_townhall - community_centre_in_2000m_radius,num_of_police_in_2000m_radius,num_of_place_of_worship_in_2000m_radius,num_of_bank_in_2000m_radius,num_of_atm_in_2000m_radius
0,1,NaN,NaN,NaN,NaN,0,0,0,0,0,...,0,3,1,11,3,0,2,18,0,0
1,5,5.0,4.0,NaN,NaN,1,1,1,1,0,...,4,16,5,38,20,0,13,3,34,51
2,4,NaN,NaN,NaN,NaN,1,2,2,2,0,...,5,20,7,39,13,0,7,13,10,46
3,7,7.0,7.0,NaN,NaN,1,3,3,3,0,...,15,694,25,52,25,0,38,80,128,466
4,4,NaN,NaN,NaN,NaN,1,4,4,4,0,...,11,107,8,50,23,0,14,40,35,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170959,1,2.0,2.0,NaN,NaN,0,119,600,9,0,...,4,11,6,41,24,0,14,5,13,58
170960,6,10.0,10.0,NaN,NaN,0,231,358,11,0,...,13,86,11,66,25,0,20,15,64,144
170961,8,22.0,22.0,NaN,NaN,1,164,574,10,0,...,2,39,1,9,4,0,4,13,1,7
170962,5,NaN,NaN,NaN,NaN,1,35,125,15,0,...,5,24,6,31,10,0,2,12,10,21


In [132]:
df.to_csv('../data/process_v1/process_data_5.csv', index = False)